# homework 1

#### Dominik Pegler  
#### 2022-06-05

Due: June 05 2022

## Prepare data

In [58]:
library(ggplot2)

filelist = list.files(pattern = ".*.txt")
datalist = lapply(filelist, function(x)read.table(x, header=T)) 
df <- do.call("rbind", datalist)

df$sbj <- factor(df$sbj)

df$answerCorrect <- factor(df$answerCorrect,levels=c(0,1,99),labels=c("incorrect","correct","timeout"))

# eliminate subject 12
df <- df[df$sbj != "12",]

# All subjects have responded 320 times

# ## Data cleaning II

# eliminate participants with
# * too long reaction time (time-out)  
# * reaction time < 0.150s  
# * reaction time > 1.000s

df <- df[df$answerCorrect!="timeout" & df$reactionTime<=1 & df$reactionTime >= 0.15,]

## Split data set and compare means

# Split into incorrect and correct answers

df_correct <- df[df$answerCorrect=="correct",]
df_incorrect <- df[df$answerCorrect=="incorrect",]

## Bonus: Validity effect

# 1) per subject for all conditions 
# 2) total for all conditions

valid <- df_correct[df_correct$targetValidity=="valid",]
valid <- aggregate(valid$reactionTime, list(valid$sbj), FUN=mean) 

invalid <- df_correct[df_correct$targetValidity=="invalid",]
invalid <- aggregate(invalid$reactionTime, list(invalid$sbj), FUN=mean) 

In [59]:
# Overall mean difference

mean(invalid$x - valid$x)

[1] 0.008329527

**Simulate the design and data of your experiment (factors and number of trials per experimental condition) 1000 times to estimate the statistical power under the following conditions.**

1. Determine the statistical power of your experiment to find a validity effect of 20 ms.
2. How small can the validity effect be to achieve a test power of 80%.
3. How small can the validity effect be to achieve a test power of 90%.
4. Finds the minimum number of participants needed to find an effect of 20 ms with a test power of 90%. 
5. Finds the average minimum number of valid trials per cue condition needed to find an effect of 20 ms with a test power of 90%. To do this, you can adjust the trialChangeFactor variable to change the number of trials per participant. The variable is a multiplier, so a trialChangeFactor of 1.5 means one and a half times as many trials per participant as in your experiment, 0.5 means half as many trials, etc. The ratio of valid and invalid trials remains the same. You can calculate the average number of valid runs per person by determining this in your data set and multiplying it by the trialChangeFactor or by determining it from the simulated data.

## 1. Determine the statistical power of your experiment to find a validity effect of 20 ms.

Note: I calculate this here only for correct responses, per observation (not per subject)

In [101]:
library(pwr)

Just show the real difference

In [80]:
mean_valid <- mean(df[df$answerCorrect=="correct" & df$targetValidity=="valid", "reactionTime"])
mean_valid

[1] 0.5935154

In [81]:
mean_invalid <- mean(df[df$answerCorrect=="correct" & df$targetValidity=="invalid", "reactionTime"])
mean_invalid

[1] 0.6030484

In [112]:
cat(mean_invalid  - mean_valid, "s\n")
cat(1000*(mean_invalid  - mean_valid), "ms")

0.009533055 s
9.533055 ms

Compute cohen's and power 

In [106]:
mean_diff_ <- 0.020 # seconds

sd_pool <- sqrt((sd_valid^2 + sd_invalid^2) / 2)
cohens_d_ <- mean_diff_ / sd_pool
n_obs <- nrow(df[df$answerCorrect=="correct" & df$targetValidity=="valid", ]) + nrow(df[df$answerCorrect=="correct" & df$targetValidity=="invalid", ])

pwr.t.test(d=cohens_d_, n=n_obs)


     Two-sample t test power calculation 

              n = 4460
              d = 0.1509994
      sig.level = 0.05
          power = 0.9999999
    alternative = two.sided

NOTE: n is number in *each* group


Power is close to 100% if we use number of observations (every subject has been observed multiple times) as our sample size and we only take correct responses into account.